# Download GEE Imagery

## Step-by-step tutorial

### Import libraries

In [110]:
import ee
import math
import geemap
import sys
import pandas as pd
from multiprocessing import Process, cpu_count 
from geetools import cloud_mask

In [111]:
Map = geemap.Map()
kazbegi = ee.FeatureCollection("users/andihanzl/kazbegi");
Map.centerObject(kazbegi, 10)
Map

Map(center=[42.6252601637589, 44.544784542321295], controls=(WidgetControl(options=['position'], widget=HBox(c…

In [112]:
snow = ee.ImageCollection('MODIS/006/MOD10A1') \
            .merge(ee.ImageCollection('MODIS/006/MYD10A1')) \
            .select(['NDSI_Snow_Cover'],['Snow']) \
            .filterBounds(kazbegi) \
            .filter(ee.Filter.date('2000-01-01', '2003-12-31'))

res = snow.first().projection()

snow = snow.map(lambda image: image.reproject(res).clip(kazbegi))

inBands = ee.List(['LST_Day_1km','LST_Night_1km','Day_view_time','Night_view_time']);
outBands = ee.List(['day','night','daytime','nighttime']); 

wr = ee.ImageCollection('MODIS/006/MOD11A1') \
        .filter(ee.Filter.date('2021-03-01', '2022-01-01')) \
        .select(inBands,outBands) \
        .sort('system:time_start')

wr = wr.map(lambda image: image.resample('bicubic').reproject(res).clip(kazbegi))

listOfImages = wr.toList(wr.size());

def wrterra(img):
    index = listOfImages.indexOf(img);
    img = ee.Image(img);
    previousIndex = ee.Algorithms.If(index.eq(0), index, index.subtract(1));
    previousImage = ee.Image(listOfImages.get(previousIndex));
    # select day temperature
    day = img.expression('day * 0.02 - 273.15', {'day': img.select('day')});
    # select night temperature
    night = previousImage.expression('night * 0.02 - 273.15', {'night': previousImage.select('night')});
    # calculate warming rate from night to day temperature
    wr = ee.Image(day.subtract(night).copyProperties(img, ["system:time_start"]));
    return img.addBands(wr.rename('wr'));

wr = ee.ImageCollection(listOfImages.map(lambda image: wrterra(image)));

#wr = ee.ImageCollection.fromImages(wr.iterate(wrterra,second)); 
wr = wr.sort('system:time_start')

def getCover(image, aoi, scale):
    #calculate the number of inputs
    totPixels = ee.Number(image.unmask(1).reduceRegion(reducer = ee.Reducer.count(),
                                                       scale = scale,
                                                       geometry = aoi)
                          .values().get(0))
    
    #calculate the actual amount of pixels inside the aoi
    actPixels = ee.Number(image.reduceRegion(reducer = ee.Reducer.count(),
                                             scale = scale,
                                             geometry = aoi)
                          .values().get(0))
    #calculate the perc of cover
    percCover = actPixels.divide(totPixels).multiply(100).round()    
    #number as output
    return image.set('percCover', percCover);

wr = wr.map(lambda image: getCover(image, kazbegi, 500))
wr = wr.filter(ee.Filter.gt('percCover', 0.000000000000001))
wr = wr.select('wr').sort("system:time_start")

count = wr.size()
print('Count: ', count.getInfo())

vis_params = {
    'bands': ['wr'],
      'min': -20,
      'max': 30,
      'palette': ['blue', 'red']
}

#Map.addLayer(snow_mask, vis_params, "Snow Mask")
Map.addLayer(wr.first(), vis_params, "Warming Rate")

Count:  32


In [113]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 17:20:38 2020

@author: kgn
"""
import sys
sys.path.append("C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/")
print(sys.path)
import ee
import numpy as np
from osgeo import gdal
from osgeo import osr
    


# Initialize the Earth Engine library.
ee.Initialize()


# fill all NA-values with a fill values of -999999 
def na_to_num(image):
    image_wo_na = image.unmask(-999999)
    return image_wo_na

# mosaic all images on the same date
def helper_function(d, imcol):
    d = ee.Date(d)
    im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
    return im.set("system:time_start", d.millis(), "system:id", d.format("YYYY-MM-dd"))

def mosaic(imcol):
    imlist = imcol.toList(imcol.size()) 
    unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()
    mosaic_imlist = unique_dates.map(lambda d: helper_function(d, imcol))
    return ee.ImageCollection(mosaic_imlist)

def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("yyyyMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

# convert the gee objects into numpys and then into Geotiffs
def gee_to_numpy(img, bounds, poly, indexname):
    path_orig = 'original_files_'+indexname+'/'
    # get the lat lon and add the ndvi
    latlon = ee.Image.pixelLonLat().addBands(img)

    # apply reducer to list
    latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=bounds,
      maxPixels=1e8,
      scale=500);
    
    # get data into three different arrays 
    data = np.array((ee.Array(latlon.get(indexname)).getInfo()))
    lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
    lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))
    
   
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
     
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)
     
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]
     
    # create an array with dimensions of image
    arr= np.zeros([nrows, ncols], np.float32) 
  
   # fill the array with values
    counter =0
    for r in range(0,len(arr),1):
        for p in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[r] and lons[counter] == uniqueLons[p] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-r,p] = data[counter] # we start from lower left corner
                    
    # set -999999 to Na
    arr[arr == -999999] = 'nan' 
    
 #   if sat == 'l' or sat == 'm': # create Geotiffs from the landsat and MODIS images at the date 0 
        #SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
     
    # set the coordinate system
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
     
    # set driver
    driver = gdal.GetDriverByName('GTiff')
    
    timedate = ee.Date(img.get('system:time_start')).format('yyyyMMdd').getInfo()
    outputDataset = driver.Create(path_orig+timedate+poly+".tif", ncols,nrows, 1,gdal.GDT_Float32)
     
    # add some metadata
    outputDataset.SetMetadata( {'time': str(timedate)} )
     
    outputDataset.SetGeoTransform(transform)
    outputDataset.SetProjection(target.ExportToWkt())
    outputDataset.GetRasterBand(1).WriteArray(arr)
    outputDataset.GetRasterBand(1).SetNoDataValue(-999999)
    outputDataset = None
    return(outputDataset, arr)
   # else:
   #     return(arr)

['C:\\Users\\andre\\OneDrive\\Dokumente\\Masterarbeit\\GEE\\Download_Leonie', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\python37.zip', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\DLLs', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm', '', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\Pythonwin', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\andre\\.ipython', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/', 'C:/Users/andre/OneDriv

In [114]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 10 09:19:52 2020

@author: kgn
"""

import ee
from geetools import cloud_mask
import pandas as pd
#import math


# Initialize the Earth Engine library.
ee.Initialize()

def download_imagery(collection, polygon, polygonname, number_of_processes):
   # modis = modis_col
    bounds = polygon
    #bounds = ee.FeatureCollection("users/leonieanina93/HessenBuff_Sued")
    
    # set the NAs to the numeric fillvalue of -999999
    #modis_wo_na = (modis_final.map(na_to_num))
    col_wo_na = (collection.map(na_to_num))
      
    # create a list from the collection  
    #modis_list = modis_wo_na.toList(modis_wo_na.size())
    col_list = col_wo_na.toList(col_wo_na.size())
    
    # get date from image and create a datelist
    #modis_dates = ymdList(modis_wo_na)
    col_dates = ymdList(col_wo_na)
          
    #create dataframe with dates from images from Landsat and Modis that are used for STARM-fusion   
    #modis_df = pd.DataFrame(
     #      {'modis': modis_dates
      #     })
       
    sat_df = pd.DataFrame(
           {#'modis': landsat_dates,
            'wr': col_dates
           })
    
  #  sat_df = pd.merge(modis_df, landsat_df, how='left')
    poly = str(polygonname)  #create a polygon label

    sat_df.to_csv(r'.\datetables_WR\polygon_' + poly + '20152021.csv')
       
    
  #  x = -1
    for j in range(len(sat_df)):
        #print(j)
       # mod_num = int(sat_df.loc[j, "index"])
        #print(mod_num)
       # path_orig = 'original_files/'
        
        #if math.isnan(float(sat_df.loc[j, "landsat"])) == False:
          #  if math.isnan(float(sat_df.loc[j, "landsat"])) == False:
             #  x = sat_df.iloc[j]['landsat']
              #  x = j
          #      x += 1
                #print(x)
                img_l = ee.Image(col_list.get(j)).clip(bounds)
               # print('Print Landsat tif')
                col_tif, arr_l = gee_to_numpy(img_l, bounds, poly, 'wr')
                
      #  else:
      #          img_m = ee.Image(modis_list.get(j)).clip(bounds)
       #         modis_tif, arr_m = gee_to_numpy(img_m, 'm', bounds, poly, 'NDVI')

In [115]:
download_imagery(wr, kazbegi, 'terra', 1)